# Title basics : extraction only for movies and TV  movies

In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime

In [2]:
url = 'https://datasets.imdbws.com/title.basics.tsv.gz'


In [3]:
chunk_size = 1000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [4]:
cleaned_chunks = []
for chunk in iterator:

    if 'endYear' in chunk.columns:
        chunk.drop(columns=['endYear'], inplace=True)      #drop endYear
    chunk.drop(columns=['isAdult','originalTitle','runtimeMinutes'], inplace=True) #drop unused columns

    if 'genres' in chunk.columns:
        chunk.dropna(subset=['genres'], inplace=True)        #suppriner les colonnes ou le genre est nulls

    if 'runtimeMinutes' in chunk.columns:
        chunk['runtimeMinutes'] = pd.to_numeric(chunk['runtimeMinutes'], errors='coerce')      #convertir les valeurs de runtimeMinutes en numerique
        chunk.dropna(subset=['runtimeMinutes'], inplace=True)        #supprimer les lignes ou runtimeLinutes est null
        
    cleaned_chunks.append(chunk)

In [5]:
full_clean_basic_title = pd.concat(cleaned_chunks, ignore_index=True)      #concaterner mes chunks pour avoir un full dataset clean
print(full_clean_basic_title.info())
print(full_clean_basic_title.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10678001 entries, 0 to 10678000
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   tconst        object 
 1   titleType     object 
 2   primaryTitle  object 
 3   startYear     float64
 4   genres        object 
dtypes: float64(1), object(4)
memory usage: 407.3+ MB
None
          startYear
count  9.345547e+06
mean   2.006254e+03
std    1.988354e+01
min    1.874000e+03
25%    2.002000e+03
50%    2.013000e+03
75%    2.019000e+03
max    2.031000e+03


In [6]:
movie_types = ['movie', 'tvMovie']
df_title_basics_movies = full_clean_basic_title[full_clean_basic_title['titleType'].isin(movie_types)]
display(df_title_basics_movies)

df_title_basics_movies.info()

,tconst,titleType,primaryTitle,startYear,genres
8,tt0000009,movie,Miss Jerry,1894.0,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897.0,"Documentary,News,Sport"
569,tt0000574,movie,The Story of the Kelly Gang,1906.0,"Action,Adventure,Biography"
586,tt0000591,movie,The Prodigal Son,1907.0,Drama
609,tt0000615,movie,Robbery Under Arms,1907.0,Drama
...,...,...,...,...,...
10677920,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,2007.0,Documentary
10677925,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,2015.0,Drama
10677931,tt9916706,movie,Dankyavar Danka,2013.0,Comedy
10677941,tt9916730,movie,6 Gunn,2017.0,Drama


<class 'pandas.core.frame.DataFrame'>
Index: 754983 entries, 8 to 10677951
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tconst        754983 non-null  object 
 1   titleType     754983 non-null  object 
 2   primaryTitle  754981 non-null  object 
 3   startYear     656434 non-null  float64
 4   genres        754983 non-null  object 
dtypes: float64(1), object(4)
memory usage: 34.6+ MB


In [7]:
df_title_basics_movies['titleType'].unique()

array(['movie', 'tvMovie'], dtype=object)

In [98]:
df_title_basics_movies = df_title_basics_movies.dropna(subset='primaryTitle').reset_index(drop=True)
df_title_basics_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754981 entries, 0 to 754980
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tconst        754981 non-null  object 
 1   titleType     754981 non-null  object 
 2   primaryTitle  754981 non-null  object 
 3   startYear     656432 non-null  float64
 4   genres        754981 non-null  object 
dtypes: float64(1), object(4)
memory usage: 28.8+ MB


In [99]:
df_title_basics_movies[df_title_basics_movies['primaryTitle'].str.contains('Oriana')]


,tconst,titleType,primaryTitle,startYear,genres
62976,tt0089739,movie,Oriana,1985.0,"Drama,Mystery,Romance"
259070,tt11079080,tvMovie,Illuminate - Oriana Fallaci,2019.0,"Biography,Documentary"
455173,tt22497614,movie,Oriana Rises,2022.0,Horror
553132,tt32188222,movie,A MAN (Oriana Fallaci Book Adaptation),NaN,Romance
583844,tt3527790,movie,Oriana: I'll Die Standing on My Feet,NaN,Drama
584943,tt3558992,tvMovie,L'Oriana,2015.0,Biography


In [103]:
# df_title_basics_movies.to_csv('df_title_basics_movies.csv', sep=';', index=False)
# !cp subset_moviessep.csv "drive/My Drive/"

### TMDB:On récupere les données complémentaires tmdb pour le productions_countries

Donwloaded csv from link : https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies
Then extraction of relevant columns


In [9]:
#https://www.nro.net/list-of-country-codes-in-the-lacnic-region/
country_code_list = ['AR','AW','BZ','BO','BR','CL','CO',
                     'CR','CU','DO','EC','SV','FK','GF',
                     'GT','GY','HT','HN','MX','AN','NI',
                     'PA','PY','PE','GS','SR','TT','UY','VE',
                     #'FR'
                     ]
###################################""
#
country_names = ['Argentina','Aruba','Bolivia','Brasil','Chile','Colombia',
                     'Costa Rica','Cuba','Dominican Republic','Ecuador','El Salvador',
                     'Guatemala',
                     'Guyana','Haiti','Honduras','Mexico',
                     'Nicaragua',
                     'Panama','Peru','Uruguay','Venezuela',
                     #'FR'
                     ]

In [10]:

url= "TMDB_movie_dataset_v11.csv"
df_tmdb = pd.read_csv(url,low_memory=False)
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118015 entries, 0 to 1118014
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1118015 non-null  int64  
 1   title                 1118002 non-null  object 
 2   vote_average          1118015 non-null  float64
 3   vote_count            1118015 non-null  int64  
 4   status                1118015 non-null  object 
 5   release_date          939457 non-null   object 
 6   revenue               1118015 non-null  int64  
 7   runtime               1118015 non-null  int64  
 8   adult                 1118015 non-null  bool   
 9   backdrop_path         298523 non-null   object 
 10  budget                1118015 non-null  int64  
 11  homepage              118817 non-null   object 
 12  imdb_id               597826 non-null   object 
 13  original_language     1118015 non-null  object 
 14  original_title        1118002 non-

In [106]:
df_tmdb.columns


Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [11]:
df_tmdb['production_countries'].unique()

array(['United Kingdom, United States of America',
       'United States of America, United Kingdom',
       'United States of America', ...,
       'Greece, Mexico, United States of America', 'Switzerland, Tunisia',
       'Bulgaria, Germany, United States of America'], dtype=object)

In [12]:
#define df_tmdb_latam
df_tmdb_latam = pd.DataFrame()
#columns we want to keep
columns_to_keep =['imdb_id', 'popularity', 'poster_path', 'production_countries']
df_tmdb = df_tmdb.loc[:,columns_to_keep]
#on ne garde que les pays  dans country_code_list
# et on identifie les pays pour lesquels on n'a aucun film

#drop na in df_tmdb['production_countries'] #remove rows with missing production country
df_tmdb.dropna(subset=['production_countries'], inplace=True)       
for country_name in  country_names:
  mask = df_tmdb['production_countries'].str.contains(country_name)
  #concat
  if (df_tmdb_latam.empty):
    df_tmdb_latam = df_tmdb[mask].copy()
  else:
    df_tmdb_latam = pd.concat([df_tmdb_latam, df_tmdb[mask]])
df_tmdb_latam.reset_index(drop=True, inplace= True)
df_tmdb_latam.info()


df_tmdb_latam.head(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31303 entries, 0 to 31302
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               20145 non-null  object 
 1   popularity            31303 non-null  float64
 2   poster_path           24969 non-null  object 
 3   production_countries  31303 non-null  object 
dtypes: float64(1), object(3)
memory usage: 978.3+ KB


,imdb_id,popularity,poster_path,production_countries
0,tt3011894,15.063,/pvSty5MCAxdj8LlAEXCDfM1YNOl.jpg,"Argentina, Spain"
1,tt1305806,17.242,/dkeAwfZzwL3WvToydE3CXiY80E0.jpg,"Argentina, Spain"
2,tt0168629,18.067,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,"Argentina, Denmark, Finland, France, Germany, ..."


In [13]:
df_tmdb_latam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31303 entries, 0 to 31302
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               20145 non-null  object 
 1   popularity            31303 non-null  float64
 2   poster_path           24969 non-null  object 
 3   production_countries  31303 non-null  object 
dtypes: float64(1), object(3)
memory usage: 978.3+ KB


In [14]:
df_tmdb_latam['production_countries'].unique()

array(['Argentina, Spain',
       'Argentina, Denmark, Finland, France, Germany, Iceland, Italy, Netherlands, Norway, Sweden, United Kingdom, United States of America',
       'Argentina, Chile, France, Germany, Peru, United Kingdom, United States of America',
       ..., 'Canada, France, Venezuela', 'France, Lebanon, Venezuela',
       'Portugal, Spain, Venezuela'], dtype=object)

In [16]:

#Merge
#inner join pour garder uniquement les ratings pour les films qui sont dans le df

df_merged_movies= pd.merge(df_title_basics_movies,df_tmdb_latam, how= 'inner',left_on='tconst',right_on= 'imdb_id')
#drop imdb_id
df_merged_movies.drop(columns=['imdb_id'],inplace=True)
df_merged_movies.reset_index(drop=True, inplace=True)
# del(df_tmdb_latam)
# del(df_title_basics_movies)
#format year as integer
df_merged_movies.dropna(subset=['startYear'],inplace=True)
df_merged_movies['startYear'] = df_merged_movies['startYear'].astype(int)

display(df_merged_movies.head(3))
display(df_merged_movies.info())

# df_merged_movies.drop(columns= ['titleId','region'],inplace=True)
# df_merged_movies.info()

# #garder le titre en FR (title) s'il n'est pas null, sinon primary title
# df_merged_movies['title'] = df_merged_movies['title'].fillna(df_merged_movies['primaryTitle'])
# df_merged_movies.info()


,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico
1,tt0007638,movie,El amor que huye,1917,Drama,1.400,NaN,Mexico
2,tt0007646,movie,El apóstol,1917,"Animation,Comedy,Drama",0.840,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,Argentina


<class 'pandas.core.frame.DataFrame'>
Index: 14933 entries, 0 to 15226
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                14933 non-null  object 
 1   titleType             14933 non-null  object 
 2   primaryTitle          14933 non-null  object 
 3   startYear             14933 non-null  int64  
 4   genres                14933 non-null  object 
 5   popularity            14933 non-null  float64
 6   poster_path           12810 non-null  object 
 7   production_countries  14933 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.0+ MB


None

In [100]:
df_merged_movies[df_merged_movies['primaryTitle']=='Oriana']

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries
1513,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France"


## Title akas - Maybe not needed if we keep original title only, but brazilian movies !
(traduction des titres en Espagnol ? plutot qu'en français? ou les deux)
Il ya très peu de titres traduits en anglais 3697/14993

In [19]:
url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

chunk_size = 10000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

cleaned_chunks = []
for chunk in iterator:

    if 'attributes' in chunk.columns:
        chunk.drop(columns=['attributes'], inplace=True)      #drop attributes

    if 'types' in chunk.columns:
        #Peggi
        chunk.drop(chunk[chunk['types']!='imdbDisplay'].index, inplace=True)      #drop rows with types different from imdbDisplay
         ##
        chunk.drop(columns=['types'], inplace=True)      #drop types

    if 'isOriginalTitle' in chunk.columns:
        chunk.drop(chunk[chunk['isOriginalTitle'] == 1].index, inplace=True)      #drop rows with isOriginalTitle

    if 'region' in chunk.columns:
        #keep all raws whose region is in country_code_list
        region_list = ['US' # we keep only the title used in the US
                    #     'AR','AW','BZ','BO','BR','CL','CO',
                    #   'CR','CU','DO','EC','SV','FK','GF',
                    #   'GT','GY','HT','HN','MX','AN','NI',
                    #   'PA','PY','PE','GS','SR','TT','UY','VE'
                      ]
        chunk = chunk.loc[chunk['region'].isin(region_list)]

    cleaned_chunks.append(chunk)

#on concatene
full_clean_title_akas = pd.concat(cleaned_chunks, ignore_index=True)

In [20]:
full_clean_title_akas['region'] = full_clean_title_akas['region'].astype(str)
full_clean_title_akas = full_clean_title_akas.drop(columns=['ordering','language','isOriginalTitle'])
full_clean_title_akas.head()

,titleId,title,region
0,tt0000001,Carmencita,US
1,tt0000003,Poor Pierrot,US
2,tt0000005,Blacksmith Scene,US
3,tt0000006,Chinese Opium Den,US
4,tt0000007,Corbett and Courtney Before the Kinetograph,US


In [21]:
#Merge
#inner join pour garder tous les films qui sont dans le df
df_merged_movies_2= pd.merge(df_merged_movies,full_clean_title_akas, how= 'left',left_on='tconst',right_on= 'titleId')
#drop title_id
df_merged_movies_2.drop(columns=['titleId'],inplace=True)
df_merged_movies_2.reset_index(drop=True, inplace=True)
del(full_clean_title_akas)

display(df_merged_movies_2.head(3))
display(df_merged_movies_2.info())

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN
1,tt0007638,movie,El amor que huye,1917,Drama,1.400,NaN,Mexico,NaN,NaN
2,tt0007646,movie,El apóstol,1917,"Animation,Comedy,Drama",0.840,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,Argentina,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                14994 non-null  object 
 1   titleType             14994 non-null  object 
 2   primaryTitle          14994 non-null  object 
 3   startYear             14994 non-null  int64  
 4   genres                14994 non-null  object 
 5   popularity            14994 non-null  float64
 6   poster_path           12869 non-null  object 
 7   production_countries  14994 non-null  object 
 8   title                 3697 non-null   object 
 9   region                3697 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.1+ MB


None

In [22]:
df_merged_movies_2.isna().sum()

tconst                      0
titleType                   0
primaryTitle                0
startYear                   0
genres                      0
popularity                  0
poster_path              2125
production_countries        0
title                   11297
region                  11297
dtype: int64

## On récupère title principals 
(actors and actresses, writer, director)

In [32]:
url = 'https://datasets.imdbws.com/title.principals.tsv.gz'
chunk_size = 10000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [33]:
cleaned_chunks = []
for chunk in iterator:
    # numeric_cols = chunk.select_dtypes(include=['number']).columns    #selectionner les colonne numerique
    if 'job' in chunk.columns:
        chunk.drop(columns=['job'], inplace=True)      #drop job

    #on supprime la colonne knownForTitles
    chunk = chunk.loc[(chunk['category'] =='actor') | (chunk['category']== 'actress') | (chunk['category']== 'director')].copy()
    chunk.drop(columns=['ordering','characters'], inplace = True) #we keep 'category'    
    #############
    cleaned_chunks.append(chunk)
    del(chunk)

In [34]:

subset_actors_directors = pd.concat(cleaned_chunks, ignore_index=True)      #concaterner mes chunks pour avoir un full dataset actors clean
print(subset_actors_directors.duplicated().value_counts())
#On supprime les doublons
subset_actors_directors= subset_actors_directors.drop_duplicates(subset=None, keep="first", inplace=False)
subset_actors_directors.reset_index(drop=True, inplace=True)

subset_actors_directors.info(verbose=True, show_counts=True)

False    43269077
True      1450537
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43269077 entries, 0 to 43269076
Data columns (total 3 columns):
 #   Column    Non-Null Count     Dtype 
---  ------    --------------     ----- 
 0   tconst    43269077 non-null  object
 1   nconst    43269077 non-null  object
 2   category  43269077 non-null  object
dtypes: object(3)
memory usage: 990.4+ MB


In [35]:
subset_actors_directors.head()

,tconst,nconst,category
0,tt0000001,nm0005690,director
1,tt0000002,nm0721526,director
2,tt0000003,nm0721526,director
3,tt0000004,nm0721526,director
4,tt0000005,nm0443482,actor


In [36]:
# inner join? it seems enough as we only want movies with actors ;p
df_merged_movies_3 = pd.merge(df_merged_movies_2,subset_actors_directors, how= 'inner',on='tconst')

# del(subset_actors_directors)

df_merged_movies_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129618 entries, 0 to 129617
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                129618 non-null  object 
 1   titleType             129618 non-null  object 
 2   primaryTitle          129618 non-null  object 
 3   startYear             129618 non-null  int64  
 4   genres                129618 non-null  object 
 5   popularity            129618 non-null  float64
 6   poster_path           112913 non-null  object 
 7   production_countries  129618 non-null  object 
 8   title                 32949 non-null   object 
 9   region                32949 non-null   object 
 10  nconst                129618 non-null  object 
 11  category              129618 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 11.9+ MB


In [37]:

df_merged_movies_3.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,category
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0220230,actress
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0846016,actor
2,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0704415,actor
3,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0749618,actress
4,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0035999,actor


## Name basics

In [38]:
url = 'https://datasets.imdbws.com/name.basics.tsv.gz'
df_name_basicschunk_size = 1000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [39]:
cleaned_chunks = []

for chunk in iterator:
    if 'primaryName' in chunk.columns:
        chunk.dropna(subset=['primaryName'], inplace=True)

    # if 'primaryProfession' in chunk.columns:
        # chunk.dropna(subset=['primaryProfession'], inplace=True)
    chunk.drop(['knownForTitles','deathYear','birthYear','primaryProfession'], axis=1, inplace=True)
    # chunk['birthYear'] = pd.to_datetime(chunk['birthYear'], format='%Y', errors='coerce')

    cleaned_chunks.append(chunk)

In [40]:
df_name_basics = pd.concat(cleaned_chunks, ignore_index=True)
display(df_name_basics.head(10))
print(df_name_basics.info())
print(df_name_basics.describe())

,nconst,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
5,nm0000006,Ingrid Bergman
6,nm0000007,Humphrey Bogart
7,nm0000008,Marlon Brando
8,nm0000009,Richard Burton
9,nm0000010,James Cagney


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13886829 entries, 0 to 13886828
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   nconst       object
 1   primaryName  object
dtypes: object(2)
memory usage: 211.9+ MB
None
           nconst primaryName
count    13886829    13886829
unique   13886829    10669548
top     nm9993719        Alex
freq            1         523


In [101]:

# jointure  name_basics avec df_merged_series, how = left : on garde toutes les lignes de merged_movies_3 ? 
df_merged_movies_4 = pd.merge(df_merged_movies_3,df_name_basics, how= 'left',on='nconst') #left_on='a_id',right_on='nconst')

df_merged_movies_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129618 entries, 0 to 129617
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                129618 non-null  object 
 1   titleType             129618 non-null  object 
 2   primaryTitle          129618 non-null  object 
 3   startYear             129618 non-null  int64  
 4   genres                129618 non-null  object 
 5   popularity            129618 non-null  float64
 6   poster_path           112913 non-null  object 
 7   production_countries  129618 non-null  object 
 8   title                 32949 non-null   object 
 9   region                32949 non-null   object 
 10  nconst                129618 non-null  object 
 11  category              129618 non-null  object 
 12  primaryName           129618 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


In [103]:
df_merged_movies_4.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,category,primaryName
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0220230,actress,Mimí Derba
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0846016,actor,Julio Taboada
2,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0704415,actor,Salvador Quiroz
3,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0749618,actress,Emilia Ruiz del Castillo
4,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0035999,actor,Salvador Arnaldo


In [119]:
df_merged_movies_4[df_merged_movies_4['primaryTitle']=='Oriana']

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,category,primaryName
15888,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0920190,actress,Doris Wells
15889,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0798866,actress,Daniela Silverio
15890,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0108539,actor,Rafael Briceño
15891,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0096587,actress,Mirtha Borges
15892,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0647495,actress,Maya Oloe
15893,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0745699,actor,Philippe Rouleau
15894,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0893417,actress,Claudia Venturini
15895,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0133863,actress,Martha Canelon
15896,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0145126,actor,Luis Armando Castillo
15897,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France",NaN,NaN,nm0578572,actor,Asdrúbal Meléndez


In [43]:
df_merged_movies_4 = df_merged_movies_4.drop_duplicates(ignore_index=True)
df_merged_movies_4.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,category,primaryName
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0220230,actress,Mimí Derba
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0846016,actor,Julio Taboada
2,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0704415,actor,Salvador Quiroz
3,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0749618,actress,Emilia Ruiz del Castillo
4,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,NaN,NaN,nm0035999,actor,Salvador Arnaldo


In [ ]:
# titles = df_merged_movies_4["primaryTitle"]
# df_merged_movies_4[titles.isin(titles[titles.duplicated()])].sort_values("primaryTitle")


In [44]:
df_merged_movies_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121857 entries, 0 to 121856
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                121857 non-null  object 
 1   titleType             121857 non-null  object 
 2   primaryTitle          121857 non-null  object 
 3   startYear             121857 non-null  int64  
 4   genres                121857 non-null  object 
 5   popularity            121857 non-null  float64
 6   poster_path           105505 non-null  object 
 7   production_countries  121857 non-null  object 
 8   title                 29511 non-null   object 
 9   region                29511 non-null   object 
 10  nconst                121857 non-null  object 
 11  category              121857 non-null  object 
 12  primaryName           121857 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.1+ MB


In [110]:
df_merged_movies_4['tconst'].nunique()

13941

In [118]:
# Identify duplicates based on specified columns
duplicates = df_merged_movies_4[df_merged_movies_4.duplicated(subset=columns_to_keep, keep=False)]

# Group by the specified columns and count occurrences
duplicate_summary = duplicates.groupby(columns_to_keep).size().reset_index(name='count')

# Display the summary of duplicates
display(duplicate_summary)

,tconst,titleType,primaryTitle,title,region,startYear,genres,popularity,poster_path,production_countries,count
0,tt0013417,movie,My American Wife,My American Wife,US,1922,Drama,0.866,/bVECaNPfOJwbDyciolVFAGlG7UG.jpg,"Argentina, United States of America",11
1,tt0018503,movie,The Ghost Train,The Ghost Train,US,1927,Romance,0.600,/lC7VHgXqqOmwowPGUDhJlAlZlMt.jpg,Mexico,11
2,tt0022756,movie,¡Que viva Mexico!,¡Que viva Mexico!,US,1932,History,1.784,/dNfI9Jiv8TyYuGi3wFqVAgiIshu.jpg,"Mexico, United States of America",12
3,tt0023902,movie,Godfather Mendoza,Godfather Mendoza,US,1934,"Drama,War",1.382,/b56SuoyQWualtXkHmrVN6HROZiD.jpg,Mexico,12
4,tt0025062,movie,Two Monks,Two Monks,US,1934,"Drama,Horror,Mystery",2.017,/mv1gSN5Bz3Btoacx7T5flr9PWcg.jpg,Mexico,11
...,...,...,...,...,...,...,...,...,...,...,...
2925,tt9863150,movie,Far from Us,Far from Us,US,2019,Drama,0.609,/creIRooa3CO4iwn65zmPUTKYLey.jpg,Argentina,6
2926,tt9866220,movie,Chocolate Road,Chocolate Road,US,2021,Documentary,0.939,/3r6P6Aut28jqKKVcRigBiUJR68m.jpg,"Japan, Peru, United States of America",2
2927,tt9881630,movie,Nahuel and the Magic Book,Nahuel and the Magic Book,US,2020,"Adventure,Animation,Family",1.449,/vwWz3biJXjhJmn5WCjocGCVG3k2.jpg,"Brazil, Chile",9
2928,tt9893012,movie,The Black Minutes,The Black Minutes,US,2021,"Crime,Thriller",1.400,/gqQJJVk9zDkXZcebV4qP4A1Fl5Z.jpg,Mexico,11


In [117]:
# Get total number of duplicates
total_duplicates = duplicate_summary['count'].sum()
print(f'Total number of duplicated entries: {total_duplicates}')

Total number of duplicated entries: 31751


In [121]:
df_merged_movies_4.columns

Index(['tconst', 'titleType', 'primaryTitle', 'startYear', 'genres',
       'popularity', 'poster_path', 'production_countries', 'title', 'region',
       'nconst', 'category', 'primaryName'],
      dtype='object')

In [122]:
#regrouper par nom de film créér une colonne avec category=='actors' or 'actress'
columns_to_keep = ['tconst','titleType','primaryTitle',
                    #'title','region',
                    'startYear','genres',
                    'popularity','poster_path',
                    'production_countries'
                    ]
# Regroupez par les colonnes spécifiées et concaténez les noms des acteurs et directeurs
mask_both_actors = (df_merged_movies_4['category'] == 'actor' ) | (df_merged_movies_4['category'] == 'actress')
df_grouped = df_merged_movies_4.groupby(columns_to_keep, as_index=False).agg(
    actors=('primaryName', lambda x: ', '.join(x[mask_both_actors])),
    directors=('primaryName', lambda x: ', '.join(x[df_merged_movies_4['category'] == 'director']))
)

# del(df_merged_movies)
df_grouped.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,actors,directors
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,"Mimí Derba, Julio Taboada, Salvador Quiroz, Em...",Joaquín Coss
1,tt0007646,movie,El apóstol,1917,"Animation,Comedy,Drama",0.840,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,Argentina,,Quirino Cristiani
2,tt0007888,movie,En defensa propia,1917,Drama,0.651,/w0KeSUDAqofWLZColJTE8ktW9Yl.jpg,Mexico,"Mimí Derba, María Caballé, Julio Taboada, Cata...",Joaquín Coss
3,tt0007958,movie,Peach Blossom,1917,Drama,0.600,/5IwfGMcwBXhtGDRWJo5X5mqQPu5.jpg,Argentina,"Carlos Gardel, Ilde Pirovano, Argentino Gómez,...",Francisco Defilippis Novoa
4,tt0008613,movie,La soñadora,1917,Romance,0.901,/hKplVE8f0FGRLBaX8RLij8WPJQ9.jpg,Mexico,"Mimí Derba, Eduardo Arozamena, Nelly Fernández...","Eduardo Arozamena, Enrique Rosas"


In [123]:
df_grouped[df_grouped['primaryTitle']=='Oriana']

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,actors,directors
1295,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France","Doris Wells, Daniela Silverio, Rafael Briceño,...",Fina Torres


In [124]:
df_grouped['tconst'].nunique()

11869

In [113]:
print(df_grouped.duplicated().value_counts())

False    3146
Name: count, dtype: int64


In [125]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11873 entries, 0 to 11872
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                11873 non-null  object 
 1   titleType             11873 non-null  object 
 2   primaryTitle          11873 non-null  object 
 3   startYear             11873 non-null  int64  
 4   genres                11873 non-null  object 
 5   popularity            11873 non-null  float64
 6   poster_path           11873 non-null  object 
 7   production_countries  11873 non-null  object 
 8   actors                11873 non-null  object 
 9   directors             11873 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 927.7+ KB


In [126]:
df_grouped[df_grouped['primaryTitle']=='Oriana']

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,actors,directors
1295,tt0089739,movie,Oriana,1985,"Drama,Mystery,Romance",1.665,/uNcJZtYAriwBrCGzIVHWq26Je4F.jpg,"Venezuela, France","Doris Wells, Daniela Silverio, Rafael Briceño,...",Fina Torres


## Title ratings

In [127]:
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
df_title_ratings = pd.read_csv(url_ratings, sep='\t', compression='gzip', na_values=["\\N"])

#inner join pour garder uniquement les ratings pour les films qui sont dans le df
df_merged_popular = pd.merge(df_grouped,df_title_ratings, how= 'inner',on='tconst')

# del(df_title_ratings)
# del(df_grouped)
display(df_merged_popular.head(3))
display(df_merged_popular.info())

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,actors,directors,averageRating,numVotes
0,tt0007646,movie,El apóstol,1917,"Animation,Comedy,Drama",0.84,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,Argentina,,Quirino Cristiani,7.1,77
1,tt0007958,movie,Peach Blossom,1917,Drama,0.60,/5IwfGMcwBXhtGDRWJo5X5mqQPu5.jpg,Argentina,"Carlos Gardel, Ilde Pirovano, Argentino Gómez,...",Francisco Defilippis Novoa,6.1,31
2,tt0008662,movie,Tepeyac,1917,Drama,0.60,/eccHRe1Tt1sv1pMcnPNFOZNsZS.jpg,Mexico,"Roberto Arroyo Carrillo, Pilar L. Cotta, Beatr...","Carlos E. González, José Manuel Ramos, Fernand...",5.8,70


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9855 entries, 0 to 9854
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                9855 non-null   object 
 1   titleType             9855 non-null   object 
 2   primaryTitle          9855 non-null   object 
 3   startYear             9855 non-null   int64  
 4   genres                9855 non-null   object 
 5   popularity            9855 non-null   float64
 6   poster_path           9855 non-null   object 
 7   production_countries  9855 non-null   object 
 8   actors                9855 non-null   object 
 9   directors             9855 non-null   object 
 10  averageRating         9855 non-null   float64
 11  numVotes              9855 non-null   int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 924.0+ KB


None

In [138]:
# Identify duplicates based on specified columns
duplicates = df_merged_popular[df_merged_popular.duplicated(subset='tconst', keep=False)]

# Group by the specified columns and count occurrences
duplicate_summary = duplicates.groupby('tconst').size().reset_index(name='count')

# Display the summary of duplicates
display(duplicate_summary)

,tconst,count
0,tt0130927,2


In [141]:
df_merged_popular[df_merged_popular['primaryTitle'] == 'tt0130927']

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,actors,directors,averageRating,numVotes


In [136]:
df_merged_popular.duplicated().sum()


np.int64(0)

In [83]:
df_merged_clean= df_merged_popular.drop_duplicates(subset='primaryTitle').reset_index(drop=True)

In [84]:
df_merged_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2956 entries, 0 to 2955
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                2956 non-null   object 
 1   title                 2956 non-null   object 
 2   primaryTitle          2956 non-null   object 
 3   startYear             2956 non-null   int64  
 4   genres                2956 non-null   object 
 5   production_countries  2956 non-null   object 
 6   popularity            2956 non-null   float64
 7   poster_path           2956 non-null   object 
 8   actors                2956 non-null   object 
 9   directors             2956 non-null   object 
 10  averageRating         2956 non-null   float64
 11  numVotes              2956 non-null   int64  
 12  combined_features     2956 non-null   object 
 13  index                 2956 non-null   int64  
dtypes: float64(2), int64(3), object(9)
memory usage: 323.4+ KB


## Ecriture du df trié par popularité dans un fichier csv 

In [57]:
# df_merged_popular = df_merged_popular.sort_values(by=['averageRating','numVotes'], ascending=False)

# df_merged_popular.to_csv("grouped_movies_latam_ML.csv",index=False,sep =';')

## Preprocessing for ML

In [85]:
# we choose to use the primaryTitle rather then the title  with translation
def combine_features(row):
    return row['primaryTitle']+' '+row['genres']+' '+row['actors'] +' '+row['directors']

def create_feature_columns(df):
    df['combined_features'] = df.apply(combine_features, axis = 1)
    df.reset_index(drop=True, inplace=True)
    df['index'] = df.index
    return df


In [86]:
df_merged_pop_features = create_feature_columns(df_merged_clean)

In [87]:
df_merged_pop_features.head()


,tconst,title,primaryTitle,startYear,genres,production_countries,popularity,poster_path,actors,directors,averageRating,numVotes,combined_features,index
0,tt11373548,Undocumented Unafraid,Undocumented Unafraid,2020,Documentary,"Mexico, United States of America",0.600,/tMF32BDdk0IC5pojPHaTHwON6Fk.jpg,Milton Benitez,Arturo Pulido,9.6,15,Undocumented Unafraid Documentary Milton Benit...,0
1,tt0439292,Hidden Secret,Hidden Secret,2003,"Drama,Mystery,Thriller",Mexico,0.757,/f69BYDF9pJwxBrrjeImAfUtEECs.jpg,"Tania Arredondo, Héctor Bonilla, Rodrigo Cache...",Emilio Ramon Vidal,9.3,28,"Hidden Secret Drama,Mystery,Thriller Tania Arr...",1
2,tt1679275,Reminiscences,Reminiscences,2010,"Documentary,Family",Peru,0.600,/2nm5fmBN2zjT7rvZSZ3HQQiAQqK.jpg,,Juan Daniel Fernández Molero,9.2,53,"Reminiscences Documentary,Family Juan Daniel ...",2
3,tt2659210,Silo,Silo,2015,Biography,Argentina,0.600,/rpMgWEeCbiKWiv1v3dxGvbtHXYx.jpg,"Vane Miller, Osmar Núñez, Gastón Pauls",Leandro Bartoletti,9.2,14,"Silo Biography Vane Miller, Osmar Núñez, Gastó...",3
4,tt22177922,Mi Gallo,Mi gallo,2024,Drama,Mexico,0.000,/bdPZLYWypanuIYfCB8ioeuGuIEm.jpg,"Escalante Alejandro, Alfredo Rios El Komander,...",Miguel Molleda,9.1,71,"Mi gallo Drama Escalante Alejandro, Alfredo Ri...",4


In [88]:
titles = df_merged_pop_features["primaryTitle"]
df_merged_pop_features[titles.isin(titles[titles.duplicated()])].sort_values("primaryTitle")

,tconst,title,primaryTitle,startYear,genres,production_countries,popularity,poster_path,actors,directors,averageRating,numVotes,combined_features,index


In [89]:
df_merged_pop_features['primaryTitle'].duplicated().value_counts()

primaryTitle
False    2956
Name: count, dtype: int64

In [91]:
df_merged_pop_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2956 entries, 0 to 2955
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                2956 non-null   object 
 1   title                 2956 non-null   object 
 2   primaryTitle          2956 non-null   object 
 3   startYear             2956 non-null   int64  
 4   genres                2956 non-null   object 
 5   production_countries  2956 non-null   object 
 6   popularity            2956 non-null   float64
 7   poster_path           2956 non-null   object 
 8   actors                2956 non-null   object 
 9   directors             2956 non-null   object 
 10  averageRating         2956 non-null   float64
 11  numVotes              2956 non-null   int64  
 12  combined_features     2956 non-null   object 
 13  index                 2956 non-null   int64  
dtypes: float64(2), int64(3), object(9)
memory usage: 323.4+ KB


## Create final csv file for ML

In [92]:
df_merged_pop_features.loc[df_merged_pop_features['tconst'] == 'tt9916680']

,tconst,title,primaryTitle,startYear,genres,production_countries,popularity,poster_path,actors,directors,averageRating,numVotes,combined_features,index


In [93]:
df_merged_pop_features = df_merged_pop_features.sort_values(by=['averageRating','numVotes'], ascending=False)

df_merged_pop_features.to_csv("grouped_movies_latam_ML.csv",index=False,sep =';')

# Test

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def init_model(features_column):
  #create a new column with combined features
  # df['combined_features'] = df.apply(combine_features, axis = 1)
  # df.reset_index(drop=True, inplace=True)
  # df['index'] = df.index

  #fit and transform the count vectorizer model
  cv = CountVectorizer()
  count_matrix = cv.fit_transform(features_column)
  cosine_sim = cosine_similarity(count_matrix)
  return cosine_sim

similarity = init_model(df_merged_pop_features['combined_features'])